# Euristic approach
### Linear relaxation:
We first model a linear relaxation of the original problem

In [3]:
import mip
import networkx as nx
import matplotlib.pyplot as plt
from math import sqrt, pow
from robomarkt_0 import Cx, Cy, usable, Dc, maxdist, mindist, maxstores, Fc, Vc

'''
# reduce problem size
n = 15
Cx = Cx[0:n]
Cy = Cy[0:n]
usable = usable[0:n]
Dc = Dc[0:n]
'''

n = len(usable)
villages = range(n)

print(n)

usable = [1 if usable[i] else 0 for i in villages]

D = [[sqrt(pow(Cx[i]-Cx[j],2) + pow(Cy[i]-Cy[j], 2)) for i in villages] for j in villages]   # Euclidean distances matrix

max_trucks_n = n//maxstores + (1 if n%maxstores > 0 else 0)   #needed trucks
trucks = range(max_trucks_n)

m_relax = mip.Model()   #exact model

# VARIABLES

stores = [m_relax.add_var(var_type=mip.CONTINUOUS, lb=0, ub=1) for i in villages]    #stores[i] tells if a store is built in village i
links  = [[m_relax.add_var(var_type=mip.CONTINUOUS, lb=0, ub=1) for i in villages] for j in villages]    #links[i][j] tells if village i has store j as assigned store (a store no further than maxdist km away)
hired_trucks = [m_relax.add_var(var_type=mip.CONTINUOUS, lb=0, ub=1) for t in trucks]     #hired_trucks[t] = 1 if truck t is used
trucks_paths = [[[m_relax.add_var(var_type=mip.CONTINUOUS, lb=0, ub=1) for i in villages] for j in villages] for t in trucks]   #trucks_paths[t][i][j] = 1 if truck t uses the road between village i and village j
trucks_visits = [[m_relax.add_var(var_type=mip.CONTINUOUS, lb=0, ub=1) for i in villages] for t in trucks]   #trucks_visits[t][i] = 1 if trucks t refills store i
counters = [[m_relax.add_var(var_type=mip.CONTINUOUS) for i in villages] for t in trucks]

# CONSTRAINTS

# we can build stores only where we have the permission
for i in villages:
    m_relax.add_constr(stores[i] <= usable[i])

# a store is built in village 0 for company image reasons
m_relax.add_constr(stores[0] == 1)

# assign each village to a store
for i in villages:
    m_relax.add_constr(mip.xsum(links[i][j] for j in villages) == 1)  

# the store has to be existing
for j in villages:
    m_relax.add_constr(mip.xsum(links[i][j] for i in villages) <= n*stores[j])

# the distance between each village and the assigned store must at most maxdist
for i in villages:
    m_relax.add_constr(mip.xsum(D[i][j]*links[i][j] for j in villages) <= maxdist)

# the distance between stores has to be at least mindist
for i in villages:
    for j in [v for v in villages if v != i]:
        m_relax.add_constr(D[i][j] >= mindist*(stores[i] + stores[j] - 1))

# we define a path only for the trucks we actually use
for t in trucks:
    m_relax.add_constr(mip.xsum(trucks_paths[t][i][j] for i in villages for j in villages) <= n*n*hired_trucks[t])

for t in trucks:
    m_relax.add_constr(mip.xsum(trucks_visits[t][i] for i in villages) <= n*hired_trucks[t])

# a single truck can visit up to maxstores stores
for t in trucks:
    m_relax.add_constr(mip.xsum(trucks_visits[t][i] for i in villages) <= maxstores + 1)

# every store has to be refilled
for i in villages[1:n]:    #the base location is considered automatically refilled
    m_relax.add_constr(mip.xsum(trucks_visits[t][i] for t in trucks) == stores[i])

# the path of each truck is an Hamiltonian cicle between the chosen stores
for t in trucks:
    for i in villages:
        m_relax.add_constr(mip.xsum(trucks_paths[t][j][i] for j in [v for v in villages if v != i]) == trucks_visits[t][i])  # a single entry path
        m_relax.add_constr(mip.xsum(trucks_paths[t][i][j] for j in [v for v in villages if v != i]) == trucks_visits[t][i])  # a single exit path
        m_relax.add_constr(trucks_paths[t][i][i] == 0)   # we avoid reflexive loops

# we enforce connectivity of the cycle
for t in trucks: 
    m_relax.add_constr(counters[t][0] == 0)     
    for i in villages:
        for j in villages[1:n]:
            m_relax.add_constr(counters[t][j] <= counters[t][i] + 1 - n*(trucks_paths[t][i][j] - 1))
            m_relax.add_constr(counters[t][j] >= counters[t][i] + 1 + n*(trucks_paths[t][i][j] - 1))
        m_relax.add_constr(mip.xsum(trucks_visits[t][v] for v in villages) <= counters[t][i] + 1 - n*(trucks_paths[t][i][0] - 1))
        m_relax.add_constr(mip.xsum(trucks_visits[t][v] for v in villages) >= counters[t][i] + 1 + n*(trucks_paths[t][i][0] - 1))

# the village at index 0 is part of this path as it is the base of the company
for t in trucks:
    m_relax.add_constr(trucks_visits[t][0] == hired_trucks[t]) 

# OBJECTIVE FUNCTION

m_relax.objective = mip.minimize(mip.xsum(Dc[i]*stores[i] for i in villages) + mip.xsum(Fc*hired_trucks[t] for t in trucks) + mip.xsum(Vc*D[i][j]*trucks_paths[t][i][j] for t in trucks for i in villages for j in villages))

# 73755.17060423897 --> result for robomarkt_0 without cuts

99


We now add as many cuts as possible in order to describe a polygon as close to the convex hull of feasible solutions as possible

In [4]:
for i in villages:
    for j in villages:
        m_relax.add_constr(links[i][j] <= stores[j])

for i in villages:
    for j in villages:
        if D[i][j] > maxdist:
            m_relax.add_constr(links[i][j] == 0)

for i in villages:
    for j in [v for v in villages if v != i]:
        if D[i][j] < mindist:
            m_relax.add_constr(stores[i] + stores[j] <= 1)
'''
for t in trucks:
    for i in villages:
        for j in villages:
            m_relax.add_constr(trucks_paths[t][i][j] <= hired_trucks[t])
'''
m_relax.optimize()
lower_bound = m_relax.objective_value

print(lower_bound)

x = input(f'{lower_bound}')

eps = 1e-3
prev_low = lower_bound
lower_bound = 0
while prev_low - lower_bound > 1e3:
    for i in villages:
        if sum([D[i][j] if links[i][j].x > eps else 0 for j in villages]) > maxdist:
            m_relax.add_constr(mip.xsum(links[i][j] for j in villages) <= sum([1 if links[i][j].x > eps else 0 for j in villages]) - 1)
    for t in trucks:
        if sum([1 if trucks_visits[t][i].x > eps else 0 for i in villages]) > maxstores:
            m_relax.add_constr(mip.xsum(trucks_visits[t][i] for i in villages) <= sum([1 if trucks_visits[t][i].x > eps else 0 for i in villages]) - 1)
    m_relax.optimize()
    prev_low = lower_bound
    lower_bound = m_relax.objective_value

print(lower_bound)
#def minimal_subsets(set, weight): # returns every subset S of villages such that sum{ D[i][S_j] * links[i][S_j] } > maxdist and, with S' = S \ j' --> sum{ D[i][S_j] * links[i][S_j] } <= maxdist fro every j'



Starting solution of the Linear programming problem using Primal Simplex

Coin0506I Presolve 587995 (-183598) rows, 155432 (-104468) columns and 2263356 (-994988) elements
Clp0030I 4 infeas 105.99269, obj 39500 - mu 1000, its 52, 694 interior
Clp0030I 8 infeas 104.23697, obj 39500 - mu 333.3, its 52, 780 interior
Clp0030I 12 infeas 103.7078, obj 39500 - mu 333.3, its 52, 734 interior
Clp0030I 16 infeas 102.62443, obj 39500 - mu 111.08889, its 52, 795 interior
Clp0030I 20 infeas 101.97254, obj 39500 - mu 37.025927, its 52, 796 interior
Clp0030I 23 infeas 101.91022, obj 39500 - mu 37.025927, its 52, 793 interior
Clp0030I 26 infeas 101.62225, obj 39500 - mu 12.340741, its 52, 795 interior
Clp0030I 30 infeas 101.61823, obj 39500 - mu 12.340741, its 52, 795 interior
Clp0030I 34 infeas 101.51517, obj 39500 - mu 4.1131691, its 52, 794 interior
Clp0030I 37 infeas 101.50159, obj 39500 - mu 1.3709193, its 52, 794 interior
Clp0030I 41 infeas 101.45479, obj 39500 - mu 1.3709193, its 52, 788 interi

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

The evaluation of the relaxed model will give us a lower bound of the solution that we can use to evaluate our euristic solutions

In [ ]:
m_relax.optimize()
lower_bound = m_relax.objective_value

print(lower_bound)